# Precision, Recall, Sensibilidade, Especificidade e Curva ROC

Esses conceitos são fundamentais para avaliar o desempenho de modelos de classificação em aprendizado de máquina e estatística. Aqui está uma explicação detalhada:

---

## **1. Precision (Precisão):**
- **Definição:** Mede a proporção de predições positivas corretas em relação a todas as predições positivas feitas pelo modelo.
- **Fórmula:**  
  $$
  \text{Precision} = \frac{\text{True Positives (TP)}}{\text{True Positives (TP)} + \text{False Positives (FP)}}
  $$
- **Interpretação:**  
  Se o modelo prevê que algo pertence à classe positiva, a precisão indica a probabilidade de estar certo.  
  É útil quando o custo de um **falso positivo** é alto.

---

## **2. Recall (Revocação ou Sensibilidade):**
- **Definição:** Mede a proporção de predições positivas corretas em relação a todos os casos positivos reais.
- **Fórmula:**  
  $$
  \text{Recall} = \frac{\text{True Positives (TP)}}{\text{True Positives (TP)} + \text{False Negatives (FN)}}
  $$
- **Interpretação:**  
  Indica a capacidade do modelo de identificar corretamente os casos positivos.  
  É importante quando o custo de um **falso negativo** é alto (ex.: diagnóstico médico).

---

## **3. Sensibilidade:**
- **Definição:**  
  É outro nome para o **recall**. Mede a habilidade do modelo em identificar corretamente os verdadeiros positivos.  
  É especialmente relevante em contextos onde **não perder positivos é essencial**.

---

## **4. Especificidade:**
- **Definição:** Mede a proporção de predições negativas corretas em relação a todos os casos negativos reais.
- **Fórmula:**  
  $$
  \text{Especificidade} = \frac{\text{True Negatives (TN)}}{\text{True Negatives (TN)} + \text{False Positives (FP)}}
  $$
- **Interpretação:**  
  Indica a capacidade do modelo de evitar falsos positivos.  
  É importante em contextos onde **rotular algo como positivo incorretamente é problemático**.

---

## **5. Curva ROC (Receiver Operating Characteristic):**
- **Definição:** É um gráfico que mostra a relação entre a **sensibilidade (recall)** e a **especificidade** para diferentes limiares de decisão.
- **Eixo X:** $1 - \text{Especificidade}$ (Taxa de Falsos Positivos).  
- **Eixo Y:** Sensibilidade (Recall ou Taxa de Verdadeiros Positivos).
- **Área sob a curva (AUC):**  
  $$
  \text{AUC (Area Under the Curve)}
  $$  
  Representa a capacidade geral do modelo de separar as classes. Um valor de AUC próximo de 1 indica um bom desempenho, enquanto um valor próximo de 0,5 indica desempenho semelhante ao acaso.

- **Interpretação:**  
  - Um modelo perfeito terá uma curva que passa pelo canto superior esquerdo (100% de recall e 100% de especificidade).  
  - A curva ajuda a entender o desempenho do modelo em diferentes limiares.

---

## **Exemplo Resumido:**
Imagine um modelo que prevê se um paciente tem uma doença:  
- **TP:** Paciente doente identificado corretamente.  
- **FP:** Paciente saudável identificado como doente.  
- **TN:** Paciente saudável identificado corretamente.  
- **FN:** Paciente doente identificado como saudável.  

### Métricas:
- **Precisão:** Quantos dos pacientes identificados como doentes estão realmente doentes?  
- **Recall (Sensibilidade):** Quantos pacientes doentes o modelo conseguiu identificar?  
- **Especificidade:** Quantos pacientes saudáveis o modelo não classificou erroneamente como doentes?  
- **Curva ROC:** Avalia o equilíbrio entre sensibilidade e especificidade para diferentes cenários.

